In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cd '/content/drive/MyDrive/Colab Notebooks/Modelv2'

In [ ]:
# !unzip '/content/drive/MyDrive/Colab Notebooks/Modelv2/celeba.zip' -d '/content/drive/MyDrive/Colab Notebooks/Modelv2/celeba'

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Input, InputLayer, Lambda, Flatten, GlobalAveragePooling2D, Dropout, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import numpy as np
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from csv import reader
import tensorflow.keras.backend as K
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Model
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from keras.utils import plot_model
from keras.utils import np_utils

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

Train, validation and test files

In [3]:
full_train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Modelv2/Dataset/10000_train_images1.csv', header = None, names=['image_id', 'path'])
full_val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Modelv2/Dataset/3000_validation_images1.csv', header = None, names=['image_id', 'path'])
full_test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Modelv2/Dataset/500_test_images1.csv', header = None, names=['image_id', 'path'])

In [4]:
# labels_file = open('/content/drive/MyDrive/Colab Notebooks/Modelv2/celeba/list_attr_celeba.csv', 'r').read().splitlines() # Read the attribute file
labels_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Modelv2/celeba/list_attr_celeba.csv')
labels_file['image_id'] = labels_file['image_id'].str.replace('.jpg', '_01.png')
labels_file
labels_file.replace(-1, 0, inplace=True)

In [5]:
train_df = pd.merge(full_train_df, labels_file, on='image_id')
val_df = pd.merge(full_val_df, labels_file, on='image_id')
test_df = pd.merge(full_test_df, labels_file, on='image_id')
train_df.drop('image_id', axis = 1, inplace=True)
val_df.drop('image_id', axis = 1, inplace=True)
test_df.drop('image_id', axis = 1, inplace=True)

In [64]:
# parameters
ImgSz = 224

## Create function that takes
# f: list contains your dataset in this format [image_path /space/ label]
# b_start, b_end: starting and ending points of the batch that you want to be extracted
# OUTPUT:
# images and labels of size (b_end - b_start + 1)

def LoadData(f, b_start, b_end):
  # print(f[0])
  # random.shuffle(f)
  b_end = min(len(f), b_end)
  # print(b_end)
  
  labels = []
  images = []
  for i in range(b_start, b_end):
    dt = f[:i+1].values
   
    im = load_img(dt[i][0])  
    im = im.resize((ImgSz, ImgSz))
    x = img_to_array(im) 
   
    x = np.array(x, dtype="float") / 255.0

    images.append(x)
    labels.append(dt[i][1:])

  return images, labels

In [61]:
# Function to be given to fit_generator. This function is responsible of 
# providing fit_generator with batch of images and their labels
# INPUT:
# f: list contains your dataset in this format [image_path /space/ label]
# batch_size: number of images per batch (CNN iteration)
def imageLoader(f, batch_size):

    numOfImgs = len(f)

    # make the generator infinite  
    while True:
      
      batch_start = 0
      batch_end = batch_size
      #random.shuffle(f)
      while batch_start < numOfImgs:
          [X, Y] = LoadData(f, batch_start, batch_end)
          X = tf.convert_to_tensor(X)
          Y = tf.convert_to_tensor(Y)
          yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

          batch_start += batch_size   
          batch_end += batch_size

In [ ]:
# All layers will be freezed except last 2 hidden layers will be trained from scratch

#Loading VGG model
VGGmodel = VGG16(weights = "imagenet", include_top=False, input_shape=(224, 224, 3))
# VGGmodel.summary() # Print the network
    

vggModel = Model(VGGmodel.inputs, VGGmodel.layers[-3].output)
# vggModel.summary()

# Freeze the weight of the rest of the layers
for layer in vggModel.layers: # We want to train the last 2 layers of VGG.
    layer.trainable = False

# Check the status of the layers 
for layer in vggModel.layers:
    print(layer, layer.trainable)
    
# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(vggModel)
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
 
# Add new layers
model.add(Flatten())
model.add(Dense(4096, activation='relu')) # Put a dense layer instead of the hidden layer
model.add(Dropout(0.5)) # Put a dropout layer to prevent the overfitting
model.add(Dense(40, activation='sigmoid')) # Output layer


 
# Show a summary of the model. Check the number of trainable parameters
model.summary()
# Check the status of the layers 
for layer in model.layers:
    print(layer, layer.trainable)

58892288/58889256 [==============================] - 1s 0us/step
<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f42bd393a50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f42bd37cfd0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f42bd340f90> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f42bb36c490> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f42bb371c10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f42b92f4290> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f42bb371290> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f42b92fd510> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f42b9302b50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f42b9306bd0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f42b92fd4d0> Fals

In [ ]:
m = model.get_layer(index = 0)
print(m.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
epochs = 10
batch_size = 80

# Compile the model
model.compile(optimizer='adadelta', metrics='binary_accuracy', loss="binary_crossentropy")

# model.load_weights('/content/drive/MyDrive/Colab Notebooks/Modelv2/models/vgg_face/10-0.85')

filepath = '/content/drive/MyDrive/Colab Notebooks/Modelv2/models/' + "vgg_face_02/{epoch:02d}-{val_binary_accuracy:.2f}"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, save_weights_only=True,
    monitor="val_binary_accuracy", 
    verbose=1, 
    save_best_only=True, mode="max")

# TRAINING
model.fit(imageLoader(train_df, batch_size), steps_per_epoch=len(train_df)/batch_size, epochs = epochs, 
                    verbose=1, validation_data=imageLoader(val_df, batch_size), validation_steps=len(val_df)/batch_size, callbacks=[checkpoint_callback])

model.save('/content/drive/MyDrive/Colab Notebooks/Modelv2/models/vgg_face_02/model')


Epoch 1/10
125/125 [==============================] - 8335s 67s/step - loss: 0.6520 - binary_accuracy: 0.6341 - val_loss: 0.4412 - val_binary_accuracy: 0.8053

Epoch 00001: val_binary_accuracy improved from -inf to 0.80529, saving model to /content/drive/MyDrive/Colab Notebooks/Modelv2/models/vgg_face_02/01-0.81
Epoch 2/10
125/125 [==============================] - 8199s 66s/step - loss: 0.4704 - binary_accuracy: 0.7880 - val_loss: 0.4254 - val_binary_accuracy: 0.8131

Epoch 00002: val_binary_accuracy improved from 0.80529 to 0.81307, saving model to /content/drive/MyDrive/Colab Notebooks/Modelv2/models/vgg_face_02/02-0.81
Epoch 3/10
125/125 [==============================] - 8188s 66s/step - loss: 0.4550 - binary_accuracy: 0.7960 - val_loss: 0.4187 - val_binary_accuracy: 0.8188

Epoch 00003: val_binary_accuracy improved from 0.81307 to 0.81879, saving model to /content/drive/MyDrive/Colab Notebooks/Modelv2/models/vgg_face_02/03-0.82
Epoch 4/10
125/125 [==============================] 

Accuracy on the test dataset

In [65]:
scores = model1.evaluate_generator(imageLoader(test_df, batch_size), steps = len(test_df)/batch_size)
print('Final test accuracy:', (scores[1]*100.0))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1973: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


Final test accuracy: 84.99999642372131


Load model

In [10]:
model1 = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Modelv2/models/vgg_face/model')

In [50]:
def getTargetPred(line):
  dt = line
  im = load_img(dt[0])  
  im = im.resize((ImgSz, ImgSz))
  x = img_to_array(im) 
  #x = x.reshape((1,) + x.shape) 
  x = np.array(x, dtype="float") / 255.0
    
  img = np.array([x])
  y_pred = model1.predict(img)
  y_target = [[int(lx) for lx in dt[1:]]]
  return y_target[0], y_pred[0]

In [71]:
def PerAttributeAcc():
  print("Calculating the accuracy of each attribute.")
  print("This may take about 10 minutes...")
  accs = [0] * 40
  

  dt = test_df[:].values
  for i in range(len(test_df)):
    x = dt[i][0:]
    
    y_target, y_pred = getTargetPred(x)
    
    for index in range(len(y_pred)):
      if (y_pred[index] <= 0.5 and y_target[index] == 0) or (y_pred[index] > 0.5 and y_target[index] == 1):
        accs[index] += 1
  
  print()
  print("Accuracy of each attribute:")
  numS = len(test_df)
  attLabels = {0:"5o'Clock Shadow", 1:"ArchedEyebrows" , 2:"Attractive/not", 3:"Bags Under Eyes/not" , 
               4:"Bald/not" , 5:"Bangs/not" , 6:"Big Lips/not" , 7:"Big Nose/not", 8:"BlackHair", 9:"BlondHair", 10:"Blurry/not", 
               11:"Brown Hair", 12:"Bushy Eyebrows", 13:"Chubby", 14:"DoubleChin", 15:"Eyeglasses/not", 16:"Goatee", 17:"Gray Hair", 
               18:"Heavy Makeup", 19:"High cheekbones", 20:"Male/Female", 21:"MouthSlightlyOpen/not", 22:"Mustache/not", 23:"NarrowEyes", 
               24:"Beard/not", 25:"Oval Face", 26:"Pale skin", 27:"Pointy Nose", 28:"Receding Hairline/not", 29:"Rosy Cheeks", 30:"Sideburns", 
               31:"Smiling", 32:"Straight Hair", 33:"Wavy Hair", 34:"Wearing Earing/not", 35:"hat/not", 36:"lipstick/not", 37:"Necklace/not", 
               38:"Necktie", 39:"Young/not", }
  
  totalAcc = 0
  for inn, cl in enumerate(accs):
    totalAcc += cl/numS
    print(attLabels[inn] + ": " + str(cl/numS))
  
  print()
  print("Average Accuracy:", totalAcc/float(len(accs)))

In [72]:
print(PerAttributeAcc())

Calculating the accuracy of each attribute.
This may take about 10 minutes...

Accuracy of each attribute:
5o'Clock Shadow: 0.88
ArchedEyebrows: 0.75
Attractive/not: 0.75
Bags Under Eyes/not: 0.778
Bald/not: 0.97
Bangs/not: 0.872
Big Lips/not: 0.754
Big Nose/not: 0.786
BlackHair: 0.758
BlondHair: 0.85
Blurry/not: 1.0
Brown Hair: 0.762
Bushy Eyebrows: 0.856
Chubby: 0.936
DoubleChin: 0.946
Eyeglasses/not: 0.944
Goatee: 0.93
Gray Hair: 0.964
Heavy Makeup: 0.85
High cheekbones: 0.72
Male/Female: 0.882
MouthSlightlyOpen/not: 0.742
Mustache/not: 0.958
NarrowEyes: 0.898
Beard/not: 0.838
Oval Face: 0.7
Pale skin: 0.962
Pointy Nose: 0.696
Receding Hairline/not: 0.92
Rosy Cheeks: 0.946
Sideburns: 0.946
Smiling: 0.754
Straight Hair: 0.77
Wavy Hair: 0.762
Wearing Earing/not: 0.806
hat/not: 0.962
lipstick/not: 0.844
Necklace/not: 0.854
Necktie: 0.904
Young/not: 0.8

Average Accuracy: 0.8500000000000002
None
